# London Housing and Population Analysis (2002 - 2015 data)

## 6. Housing and Population Merged Data set creation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Compare and merge ages and prices data sets:

ages = pd.read_csv('..raw_data/population_dataset_bracketed.csv', index_col=[0])
prices = pd.read_csv('..raw_data/borough_dataset.csv')

# Confirm data set structure:
ages.info()
prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8918 entries, 0 to 8917
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ward_code  8918 non-null   object
 1   year       8918 non-null   int64 
 2   ward_name  8918 non-null   object
 3   borough    8918 non-null   object
 4   all_ages   8918 non-null   int64 
 5   minor      8918 non-null   int64 
 6   18_24      8918 non-null   int64 
 7   25_34      8918 non-null   int64 
 8   35_44      8918 non-null   int64 
 9   45_54      8918 non-null   int64 
 10  55_64      8918 non-null   int64 
 11  65+        8918 non-null   int64 
dtypes: int64(9), object(3)
memory usage: 905.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9867 entries, 0 to 9866
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           9867 non-null   object 
 1   area           9867 non-null   object 
 2   average_

In [3]:
ages['year'] = ages['year'].astype('str')
prices['date'] = pd.to_datetime(prices['date']).dt.strftime('%Y')

# match appelations:
ages['borough'] = ages['borough'].str.lower()
prices = prices.rename(columns={'date': 'year', 'area': 'borough'})
# check appelations and counts match:
boroughs1 = set(ages['borough'].unique().tolist())
boroughs2 = set(prices['borough'].unique().tolist())
shared_values = boroughs1.intersection(boroughs2)
len(boroughs1), len(boroughs2), len(shared_values)

# Group ages by borough:
borough_ages = ages.drop(columns=['ward_code', 'ward_name'])
borough_ages = borough_ages.groupby(['borough', 'year']).sum().reset_index()

# Group house sales and prices by borough and year:
annual_sales = prices.groupby(['borough', 'year'])['houses_sold'].sum().astype('int64').reset_index()
mean_annual_prices = prices.groupby(['borough', 'year'])['average_price'].mean().astype('int64').reset_index()
annual_housing = pd.merge(annual_sales, mean_annual_prices, how='left', on=['borough', 'year'])

# Create borough population and housing data set:
df = pd.merge(borough_ages, annual_housing, how='left', on=['borough', 'year'])
df.to_csv('..raw_data/pops_sales_and_prices.csv')